In [1]:
import threading
import QuantLib as ql
import numpy as np
import scipy.optimize as opt
import pandas as pd
import matplotlib.pyplot as plt
import IPython
import time
plt.style.use('ggplot')
#%matplotlib inline
def Calculator():
    def FXCalculate():
        # generator for Heston process
        def HestonPathGenerator(dates, dayCounter, process, nPaths):
            t = np.array([dayCounter.yearFraction(dates[0], d) for d in dates])
            nGridSteps = (t.shape[0] - 1) * 2
            sequenceGenerator = ql.UniformRandomSequenceGenerator(nGridSteps, ql.UniformRandomGenerator())
            gaussianSequenceGenerator = ql.GaussianRandomSequenceGenerator(sequenceGenerator)
            pathGenerator = ql.GaussianMultiPathGenerator(process, t, gaussianSequenceGenerator, False)
            paths = np.zeros(shape = (nPaths, t.shape[0]))

            for i in range(nPaths):
                multiPath = pathGenerator.next().value()
                paths[i,:] = np.array(list(multiPath[0]))

            # return array dimensions: [number of paths, number of items in t array]
            return paths
        def FXStructuredNote(valuationDate, ObservationDates, YesBarrier, NoBarrier, coupon, notional, 
                         dayCounter, process, generator, nPaths, curve):    
            dates = np.hstack((np.array([valuationDate]), ObservationDates[ObservationDates > valuationDate]))
            paths = generator(dates, dayCounter, process, nPaths)[:,1:]
            global_pv = []
            for path in paths:
                payoffPV = 0.0
                Break = False
                for date, price in zip(ObservationDates, path):
                    if Break:
                        break
                    if(date == lastObservationDate):
                        payoffPV = notional
                        Break = True
                    if(price <= YesBarrier):
                        Break = True
                        payoffPV = notional * (1+ (coupon)/2)
                    if(price >= NoBarrier):
                        payoffPV = notional
                        Break = True   

                global_pv.append(payoffPV)

            df = curve.discount(ObservationDates[-1])
            return np.mean(np.array(global_pv))*df, paths
        
        result_Button["text"] = "Calculating..."
        s1 = time.time()
        # general QuantLib-related parameters
        Date = str(txt000.get())
        Date = Date.split("/")
        valuationDate = ql.Date(int(Date[2]), int(Date[1]), int(Date[0]))
        ql.Settings.instance().evaluationDate = valuationDate
        convention = ql.ModifiedFollowing
        dayCounter = ql.Actual360()
        calendar = ql.UnitedStates()

        # FX Structured Note
        notional = 100
        spot = float(txt10.get())
        YesBarrier = 105.0
        NoBarrier = 110
        coupon = 0.03

        # Observation schedule for note # 假日不Observation
        Date = str(txt000.get())
        Date = Date.split("/")
        startDate = ql.Date(int(Date[2]), int(Date[1]), int(Date[0]))
        TTM = int(txt00.get())
        firstObservationDate = calendar.advance(startDate, ql.Period(1, ql.Days))
        lastObservationDate = calendar.advance(startDate, ql.Period(TTM, ql.Months))
        ObservationDates = np.array(list(ql.Schedule(firstObservationDate, lastObservationDate,ql.Period(1, ql.Days), 
            calendar, ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False)))#跳過假日
        
        #計算Libor Rate
        USDLIBORdates = [startDate, startDate + ql.Period(1, ql.Months), 
                         startDate + ql.Period(2, ql.Months), startDate + ql.Period(3, ql.Months), 
                         startDate + ql.Period(6, ql.Months), startDate + ql.Period(12, ql.Months)]
        USDLIBORrates = [0.0, float(txt30.get()), float(txt31.get()), float(txt32.get()), float(txt33.get()), float(txt34.get())]

        JPYLIBORdates = [startDate, startDate + ql.Period(1, ql.Months), 
                         startDate + ql.Period(2, ql.Months), startDate + ql.Period(3, ql.Months), 
                         startDate + ql.Period(6, ql.Months), startDate + ql.Period(12, ql.Months)]
        JPYLIBORrates = []

        SwapMaturity = [0, 1, 2, 3, 6, 12]
        SwapPoint = [0.0, float(txt40.get()), float(txt41.get()), float(txt42.get()), float(txt43.get()), float(txt44.get())]

        for i in range(len(SwapPoint)):
            ForwardPrice = spot + SwapPoint[i]
            try:
                JPYLIBORrates.append((1/(ForwardPrice/spot/(1 + USDLIBORrates[i]*SwapMaturity[i]/12)) - 1)*12/SwapMaturity[i])
            except:
                JPYLIBORrates.append(0.0)

        #建Term structure
        interpolation = ql.Linear()
        compounding = ql.Simple
        compoundingFrequency = ql.Annual

        USDLIBORCurve = ql.ZeroCurve(USDLIBORdates, USDLIBORrates, dayCounter, calendar, interpolation,
                                 compounding, compoundingFrequency)
        USDcurveHandle = ql.YieldTermStructureHandle(USDLIBORCurve)

        JPYLIBORCurve = ql.ZeroCurve(JPYLIBORdates, JPYLIBORrates, dayCounter, calendar, interpolation,
                                compounding, compoundingFrequency)
        JPYcurveHandle = ql.YieldTermStructureHandle(JPYLIBORCurve)
        
        # monte carlo parameters
        nPaths = int(txt20.get())
        
        # Parameters for Heston model
        v0 = float(txt50.get())
        kappa = float(txt51.get())
        theta = float(txt52.get())
        rho = float(txt53.get())
        sigma =  float(txt54.get())
        
        process1 = ql.HestonProcess(JPYcurveHandle, USDcurveHandle, 
                ql.QuoteHandle(ql.SimpleQuote(spot)), v0, kappa, theta, sigma, rho)
        process2 = ql.HestonProcess(JPYcurveHandle, USDcurveHandle, 
                ql.QuoteHandle(ql.SimpleQuote(spot*1.01)), v0, kappa, theta, sigma, rho)
        process3 = ql.HestonProcess(JPYcurveHandle, USDcurveHandle, 
                ql.QuoteHandle(ql.SimpleQuote(spot*0.99)), v0, kappa, theta, sigma, rho)
        process4 = ql.HestonProcess(JPYcurveHandle, USDcurveHandle, 
                ql.QuoteHandle(ql.SimpleQuote(spot)), v0, kappa, theta, sigma*1.0001, rho)

        FV1, p1 = FXStructuredNote(valuationDate, ObservationDates, YesBarrier, NoBarrier, coupon, notional, 
                             dayCounter, process1, HestonPathGenerator, nPaths, USDcurveHandle)
        FV2, p2 = FXStructuredNote(valuationDate, ObservationDates, YesBarrier, NoBarrier, coupon, notional, 
                             dayCounter, process2, HestonPathGenerator, nPaths, USDcurveHandle)
        FV3, p3 = FXStructuredNote(valuationDate, ObservationDates, YesBarrier, NoBarrier, coupon, notional, 
                             dayCounter, process3, HestonPathGenerator, nPaths, USDcurveHandle)
        FV4, p4 = FXStructuredNote(valuationDate, ObservationDates, YesBarrier, NoBarrier, coupon, notional, 
                             dayCounter, process4, HestonPathGenerator, nPaths, USDcurveHandle)
        
        Delta = (FV2 - FV3)/(0.02*spot)
        Gamma = (FV2 - 2*FV1 + FV3)/((0.01*spot)**2)
        Vega = (FV4 - FV1)/(sigma*0.0001)
        s2 = time.time()
        print(s2- s1, FV1, Delta, Gamma, Vega)
        result_Button["text"] = "Calculate"
        result1["text"] = "Time: " + str(round(s2 - s1, 2))
        result2["text"] = "Fair Value: " + str(round(FV1, 3))
        result3["text"] = "Delta: " + str(round(Delta, 3))
        result4["text"] = "Gamma: " + str(round(Gamma, 3))
        result5["text"] = "Vega: " + str(round(Vega, 3))
        
        if (var1.get() == 1):
            if len(p1 >= 1000):
                fig = pd.DataFrame(p1[:1000]).T.plot(legend=False).get_figure()
                fig.savefig('Paths.png')
            else:
                fig = pd.DataFrame(p1).T.plot(legend=False).get_figure()
                fig.savefig('Paths.png')
                    
    th=threading.Thread(target=FXCalculate)
    th.setDaemon(True)
    th.start() 

In [ ]:
from tkinter import *
import tkinter as tk
import tkinter.font as tkFont

window = Tk()

window.title("FX Structured Note")

window.geometry('1150x250')
width = 18

fontStyle = tkFont.Font(size=12)

lb00 = Label(window, text="Valuation Date (YYYY/MM/DD)", anchor = "w", justify=LEFT)
lb00.grid(column=0, row=0, sticky = W)
txt000 = Entry(window, width=width)
txt000.insert(END, '2013/11/14')
txt000.grid(column=1, row=0)

lb0 = Label(window, text="Time to Maturity (M)", anchor = "w", justify=LEFT)
lb0.grid(column=0, row=1, sticky = W)
txt00 = Entry(window,width=width)
txt00.insert(END, 6)
txt00.grid(column=1, row=1)

lb1 = Label(window, text="USD/JPY Spot", anchor = "w", justify=LEFT)
lb1.grid(column=0, row=2, sticky = W)
txt10 = Entry(window,width=width)
txt10.insert(END, 108)
txt10.grid(column=1, row=2)

lb2 = Label(window, text="Simulation Times", anchor = "w", justify=LEFT)
lb2.grid(column=0, row=3, sticky = W)
txt20 = Entry(window,width=width)
txt20.insert(END, 100000)
txt20.grid(column=1, row=3)

lb3 = Label(window, text="USD Libor Rate (1M, 2M, 3M, 6M, 12M)", anchor = "w", justify=LEFT)
lb3.grid(column=0, row=4, sticky = W)
txt30 = Entry(window,width=width)
txt30.insert(END, 0.0016750)
txt30.grid(column=1, row=4)
txt31 = Entry(window,width=width)
txt31.insert(END, 0.0020700)
txt31.grid(column=2, row=4)
txt32 = Entry(window,width=width)
txt32.insert(END, 0.0023854)
txt32.grid(column=3, row=4)
txt33 = Entry(window,width=width)
txt33.insert(END, 0.0035350)
txt33.grid(column=4, row=4)
txt34 = Entry(window,width=width)
txt34.insert(END, 0.0058810)
txt34.grid(column=5, row=4)

lb4 = Label(window, text="Swap Point (1M, 2M, 3M, 6M, 12M)", anchor = "w", justify=LEFT)
lb4.grid(column=0, row=5, sticky = W)
txt40 = Entry(window,width=width)
txt40.insert(END, -0.01325)
txt40.grid(column=1, row=5)
txt41 = Entry(window,width=width)
txt41.insert(END, -0.0425)
txt41.grid(column=2, row=5)
txt42 = Entry(window,width=width)
txt42.insert(END, -0.05855)
txt42.grid(column=3, row=5)
txt43 = Entry(window,width=width)
txt43.insert(END, -0.1221)
txt43.grid(column=4, row=5)
txt44 = Entry(window,width=width)
txt44.insert(END, -0.301)
txt44.grid(column=5, row=5)

lb5 = Label(window, text="Heston Model (v0, kappa, theta, rho, sigma)", anchor = "w", justify=LEFT)
lb5.grid(column=0, row=6, sticky = W)
txt50 = Entry(window,width=width)
txt50.insert(END, 0.0102401)
txt50.grid(column=1, row=6)
txt51 = Entry(window,width=width)
txt51.insert(END,  1.171979)
txt51.grid(column=2, row=6)
txt52 = Entry(window,width=width)
txt52.insert(END, 0.0141483)
txt52.grid(column=3, row=6)
txt53 = Entry(window,width=width)
txt53.insert(END, 0.128199)
txt53.grid(column=4, row=6)
txt54 = Entry(window,width=width)
txt54.insert(END, 0.336611)
txt54.grid(column=5, row=6)

result_Button = tk.Button(window, text = "Calculate", command = Calculator, width=25, height=1, font = fontStyle)
result_Button.grid(column=0, row=7)
result1 = tk.Label(window, font = fontStyle)
result1.grid(column=1, row=7)
result2 = tk.Label(window, font = fontStyle)
result2.grid(column=2, row=7)
result3 = tk.Label(window, font = fontStyle)
result3.grid(column=3, row=7)
result4 = tk.Label(window, font = fontStyle)
result4.grid(column=4, row=7)
result5 = tk.Label(window, font = fontStyle)
result5.grid(column=5, row=7)

var1 = tk.IntVar()
path_Button = tk.Checkbutton(window, text = "Show Paths (Maximum Paths Show: 1000)", variable = var1, anchor = "w", justify=LEFT)
path_Button.grid(column=0, row=8, sticky = W)

window.mainloop()

47.05969834327698 100.39712723360137 -0.24752534616704058 0.012567683787361955 72.95173215332295
